In [2]:
from pyspark.sql import SparkSession
import numpy as np
import pandas as pd
import os

In [3]:
spark = SparkSession.builder \
        .appName("Silver") \
        .master("local[*]") \
        .getOrCreate()

In [4]:
spark

In [31]:
bus_facility = spark.read.csv("/content/bus_facilities.csv", header=True)

bus_facility.show(truncate=False)

+---+---------------------+--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|Id |Bus_Name             |Facilities                                                                                                                                                                                                                                              |
+---+---------------------+--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|1  |Mạnh Hùng            |['Đèn đọc sách', 'Dép', 'Dây đai an toàn', 'Gửi kèm xe máy', 'Nước uống', 'Gối nằm', 'Búa phá kính', 'Tivi LED', 'Sạc điện thoại', 'Rèm cửa', 

In [32]:
bus_facility.printSchema()

root
 |-- Id: string (nullable = true)
 |-- Bus_Name: string (nullable = true)
 |-- Facilities: string (nullable = true)



In [33]:
import numpy as np
import pandas as pd
from pyspark.sql.functions import regexp_replace, split, col, size, array_contains


# Loại bỏ ký tự [ và ] trong cột Facilities
bus_facility = bus_facility.withColumn("Facilities", regexp_replace(col("Facilities"), "[\\[\\]']", ""))

# Chuyển STRING thành ARRAY bằng cách tách theo dấu phẩy và khoảng trắng
bus_facility = bus_facility.withColumn("Facilities", split(col("Facilities"), ", "))

# Remove rows where the Facilities array is empty or contains only empty strings
bus_facility = bus_facility.filter((size(col("Facilities")) > 0) & (~array_contains(col("Facilities"), "")))


bus_facility.show(truncate=False)

bus_facility.printSchema()


+---+---------------------+----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|Id |Bus_Name             |Facilities                                                                                                                                                                                                            |
+---+---------------------+----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|1  |Mạnh Hùng            |[Đèn đọc sách, Dép, Dây đai an toàn, Gửi kèm xe máy, Nước uống, Gối nằm, Búa phá kính, Tivi LED, Sạc điện thoại, Rèm cửa, Dàn âm thanh (Loa), Chăn đắp, Wifi, Điều hòa]                                               |
|2  |Toàn Khải            |[

#### Khúc này là xong, xử lý code dưới là lưu thành 1 bảng

In [39]:
from pyspark.sql import functions as F

# Chuyển đổi Facilities từ ARRAY thành STRUCT với index làm key
bus_facilities = bus_facility.select(
    "Bus_Name",
    *[F.col("Facilities")[i].alias(f"{i}") for i in range(len(bus_facility.select("Facilities").first()[0]))]
)

# Tạo biểu thức stack động
stack_expr = f"stack({len(bus_facilities.columns) - 1}, " + ", ".join(
    [f"'{col}', {col}" for col in bus_facilities.columns[1:]]
) + ") as (Facility_Id, value)"

# Chuyển đổi DataFrame sử dụng stack
bus_facilities_convert = (
    bus_facilities
    .selectExpr("Bus_Name", stack_expr)
    .filter(F.col("value").isNotNull())  # Lọc bỏ giá trị null
    .select("Bus_Name", "Facility_Id" )  # Lấy ra tên bus và các facility tương ứng
)

bus_facilities_convert.show(10)

+---------+-----------+
| Bus_Name|Facility_Id|
+---------+-----------+
|Mạnh Hùng|          0|
|Mạnh Hùng|          1|
|Mạnh Hùng|          2|
|Mạnh Hùng|          3|
|Mạnh Hùng|          4|
|Mạnh Hùng|          5|
|Mạnh Hùng|          6|
|Mạnh Hùng|          7|
|Mạnh Hùng|          8|
|Mạnh Hùng|          9|
+---------+-----------+
only showing top 10 rows



#### Tách ra 1 bảng mới ở khúc này

In [40]:
from pyspark.sql import Row
from pyspark.sql import functions as F

# Lấy danh sách giá trị duy nhất từ Facilities bằng cách explode
Facility_Names = (
    bus_facility
    .select(F.explode("Facilities").alias("Facility_Name"))
    .distinct()  # Loại bỏ trùng lặp
    .rdd.map(lambda row: row["Facility_Name"])  # Chuyển thành danh sách Python
    .collect()
)

# Tạo DataFrame Spark với cột `Facility_Id` và `Facility_Name`
Facility_Names = spark.createDataFrame(
    [Row(Facility_Id=i, Facility_Name=name) for i, name in enumerate(Facility_Names)]
)

# Hiển thị DataFrame
Facility_Names.show()

+-----------+--------------------+
|Facility_Id|       Facility_Name|
+-----------+--------------------+
|          0|                 Dép|
|          1|      Gửi kèm xe máy|
|          2|             Rèm cửa|
|          3|Nhân viên sử dụng...|
|          4|      Sạc điện thoại|
|          5|            Điều hòa|
|          6|              Toilet|
|          7|            Tai nghe|
|          8|              Gối ôm|
|          9|            Chăn đắp|
|         10|     Xe trung chuyển|
|         11|           Nước uống|
|         12|         Ghế massage|
|         13|  Dàn âm thanh (Loa)|
|         14|        Búa phá kính|
|         15|            Tivi LED|
|         16|        Đèn đọc sách|
|         17|                Wifi|
|         18|     Dây đai an toàn|
|         19|             Gối nằm|
+-----------+--------------------+
only showing top 20 rows

